In [2]:
!pip install -q tensorflow-model-optimization

In [16]:
import tempfile
import os

import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing

In [17]:
# Load data
student_df = pd.read_csv('../../datasets/student_mat_processed01.csv')

train_dataset = student_df.sample(frac=0.8, random_state=0)
test_dataset = student_df.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_G1_targets = train_features.pop('G1')
train_G2_targets = train_features.pop('G2')
train_G3_targets = train_features.pop('G3')

test_G1_targets = test_features.pop('G1')
test_G2_targets = test_features.pop('G2')
test_G3_targets = test_features.pop('G3')

In [18]:
# Normalizer Layer
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))

In [45]:
# Define the model architecture.
def build_and_compile_model(normalizer):
    model = keras.Sequential([
        normalizer,
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(1)
    ])

    # Train the model
    model.compile(loss='mean_absolute_error', metrics=[tf.keras.metrics.MeanAbsoluteError()],
                  optimizer=tf.keras.optimizers.Adam(0.001))
    
    return model

In [46]:
fc_model = build_and_compile_model(normalizer)
fc_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_2 (Normalizati (None, 45)                91        
_________________________________________________________________
dense_24 (Dense)             (None, 32)                1472      
_________________________________________________________________
dense_25 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_26 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 9         
Total params: 2,236
Trainable params: 2,145
Non-trainable params: 91
_________________________________________________________________


In [47]:
history = fc_model.fit(
    train_features, train_G3_targets,
    validation_split=0.2,
    verbose=1, epochs=100)

Epoch 1/100
8/8 [==============================] - 0s 17ms/step - loss: 10.1845 - mean_absolute_error: 10.1845 - val_loss: 11.1235 - val_mean_absolute_error: 11.1235
Epoch 2/100
8/8 [==============================] - 0s 4ms/step - loss: 9.9227 - mean_absolute_error: 9.9227 - val_loss: 10.8983 - val_mean_absolute_error: 10.8983
Epoch 3/100
8/8 [==============================] - 0s 2ms/step - loss: 9.7056 - mean_absolute_error: 9.7056 - val_loss: 10.7009 - val_mean_absolute_error: 10.7009
Epoch 4/100
8/8 [==============================] - 0s 4ms/step - loss: 9.4956 - mean_absolute_error: 9.4956 - val_loss: 10.4975 - val_mean_absolute_error: 10.4975
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 9.2701 - mean_absolute_error: 9.2701 - val_loss: 10.2579 - val_mean_absolute_error: 10.2579
Epoch 6/100
8/8 [==============================] - 0s 4ms/step - loss: 8.9831 - mean_absolute_error: 8.9831 - val_loss: 9.9584 - val_mean_absolute_error: 9.9584
Epoch 7/100
8/8 [====

8/8 [==============================] - 0s 3ms/step - loss: 1.6352 - mean_absolute_error: 1.6352 - val_loss: 3.2282 - val_mean_absolute_error: 3.2282
Epoch 52/100
8/8 [==============================] - 0s 3ms/step - loss: 1.6082 - mean_absolute_error: 1.6082 - val_loss: 3.2069 - val_mean_absolute_error: 3.2069
Epoch 53/100
8/8 [==============================] - 0s 4ms/step - loss: 1.5839 - mean_absolute_error: 1.5839 - val_loss: 3.1994 - val_mean_absolute_error: 3.1994
Epoch 54/100
8/8 [==============================] - 0s 1ms/step - loss: 1.5502 - mean_absolute_error: 1.5502 - val_loss: 3.2144 - val_mean_absolute_error: 3.2144
Epoch 55/100
8/8 [==============================] - 0s 3ms/step - loss: 1.5231 - mean_absolute_error: 1.5231 - val_loss: 3.2186 - val_mean_absolute_error: 3.2186
Epoch 56/100
8/8 [==============================] - 0s 2ms/step - loss: 1.5024 - mean_absolute_error: 1.5024 - val_loss: 3.1982 - val_mean_absolute_error: 3.1982
Epoch 57/100
8/8 [=======================

In [49]:
test_results = {}
_, test_results['fc_model'] = fc_model.evaluate(test_features, test_G3_targets, verbose=0)

print('Baseline mean absolute error:', test_results['fc_model'])

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(fc_model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline mean absolute error: 3.7244648933410645
Saved baseline model to: C:\Users\z0042fkb\AppData\Local\Temp\tmpr01hbf3t.h5
